In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpmath import mp, mpf, fmul, fadd, fsub, fdiv

# Constants
L = 100.0                  # Half-length of the domain in meters (standard float)
c = 1.0                    # Wave speed in m/s (standard float)
N = 201                    # Number of spatial points
dx = (2 * L) / (N - 1)     # Spatial step size for [-L, L] domain
dt = dx / c                # Time step size (CFL condition)
T = 1000.0                  # Total simulation time in seconds
x = np.linspace(-L, L, N)  # Spatial grid as a NumPy array
plot_times = [0, int((T / 2) / dt), int(T / dt)]  # Start, middle, and end time steps

# Output directory for results
output_dir = "/Users/danieltompkins/Documents/Devito-UROP-MSci/Scaled-results"
os.makedirs(output_dir, exist_ok=True)

# Analytical Solution
def analytical_solution(t, x=x, c=c, L=L):
    t_np = float(t)
    return np.cos(2 * np.pi * c * t_np / L) * np.sin(2 * np.pi * x / L)

# Function to compute solutions and errors at a given precision
def compute_wave_solution_and_errors(bit_depth):
    mp.prec = bit_depth
    u = analytical_solution(0)  # Initial condition u(x, 0) as float
    u_prev = u.copy()           # Initial velocity is zero: u_t(x, 0) = 0
    u_next = np.zeros(N, dtype=np.float64)  # Placeholder for the next time step
    u_num_at_t = {}
    errors = []

    for n in range(int(T / dt) + 1):
        t = n * dt
        if n in plot_times:
            u_num_at_t[n] = u.copy()

        coeff = fdiv(fmul(mpf(c), mpf(dt)), mpf(dx))
        coeff_squared = fdiv(coeff, coeff)

        for i in range(1, N - 1):
            term2 = fadd(fsub(mpf(u[i + 1]), fmul(mpf(2), mpf(u[i]))), mpf(u[i - 1]))
            update = fmul(coeff_squared, term2)
            u_next[i] = float(fadd(fsub(fmul(mpf(2), mpf(u[i])), mpf(u_prev[i])), update))
        u_next[0] = 0.0
        u_next[-1] = 0.0
        u_prev = u.copy()
        u = u_next.copy()
        u_analytic = analytical_solution(t)
        max_error = max(abs(u[i] - u_analytic[i]) for i in range(N))
        errors.append((t, max_error))

    return u_num_at_t, errors

# Function to generate and save plots and data
def save_results(bit_depth, u_num_at_t, errors):
    # FFT Plot
    u_start = u_num_at_t[plot_times[0]]
    u_final = u_num_at_t[plot_times[2]]
    fft_start = np.fft.fft(u_start)
    fft_final = np.fft.fft(u_final)
    freqs_start = np.fft.fftfreq(len(u_start), d=dt)
    freqs_final = np.fft.fftfreq(len(u_final), d=dt)
    positive_freqs_start = freqs_start[freqs_start >= 0]
    positive_freqs_final = freqs_final[freqs_final >= 0]
    fft_magnitude_start = np.abs(fft_start)[freqs_start >= 0]
    fft_magnitude_final = np.abs(fft_final)[freqs_final >= 0]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.plot(positive_freqs_start, fft_magnitude_start, label="FFT Start")
    ax1.set_title("Frequency Spectrum at t0")
    ax1.set_xlabel("Frequency (Hz)")
    ax1.set_ylabel("Amplitude")
    ax1.grid(True)
    ax2.plot(positive_freqs_final, fft_magnitude_final, label="FFT End", color="red")
    ax2.set_title("Frequency Spectrum at tN")
    ax2.set_xlabel("Frequency (Hz)")
    ax2.set_ylabel("Amplitude")
    ax2.grid(True)
    fig.suptitle(f"FFT (Precision {bit_depth} bits)")
    fft_file = os.path.join(output_dir, f"fft_precision_{bit_depth}.png")
    plt.savefig(fft_file)
    plt.close()

    # Error Plot
    times, max_errors = zip(*errors)
    plt.figure(figsize=(8, 6))
    plt.plot(times, max_errors, label="Max Absolute Error")
    plt.xlabel("Time (s)")
    plt.ylabel("Max Error")
    plt.title(f"Max Error Over Time (Precision {bit_depth} bits)")
    plt.grid(True)
    error_file = os.path.join(output_dir, f"error_precision_{bit_depth}.png")
    plt.savefig(error_file)
    plt.close()

    # Save error data to CSV
    csv_file = os.path.join(output_dir, f"errors_precision_{bit_depth}.csv")
    with open(csv_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Time", "Max Error"])
        writer.writerows(errors)

# Main Loop Over Bit Depths
for bit_depth in range(23, 3, -1):  # Loop from 23 bits to 4 bits
    u_num_at_t, errors = compute_wave_solution_and_errors(bit_depth)
    save_results(bit_depth, u_num_at_t, errors)

print(f"Results saved in '{output_dir}' directory.")


Results saved in '/Users/danieltompkins/Documents/Devito-UROP-MSci/Scaled-results' directory.


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpmath import mp, mpf, fmul, fadd, fsub, fdiv

# Constants
L = 100.0                  # Half-length of the domain in meters (standard float)
c = 1.0                    # Wave speed in m/s (standard float)
N = 201                    # Number of spatial points
dx = 2 * L / (N - 1)          # Spatial step size for [-L, L] domain
dt = dx / c             # Time step size (CFL condition)
T = 1000.0                 # Total simulation time in seconds
x = np.linspace(-L, L, N)  # Spatial grid as a NumPy array
plot_times = [0, int((T / 2) / dt), int(T / dt)]

# Analytical Solution
def analytical_solution(t, x=x, c=c, L=L):
    t_np = float(t)
    return np.cos(2 * np.pi * c * t_np / L) * np.sin(2 * np.pi * x / L)

# Function to compute the solution for a given precision
def compute_wave_solution_and_plot(bit_depth, output_dir="/Users/danieltompkins/Documents/Devito-UROP-MSci/Scaled-results/solutions"):
    mp.prec = bit_depth
    u = analytical_solution(0)
    u_prev = u.copy()
    u_next = np.zeros(N, dtype=np.float64)
    u_num_at_t = {}

    for n in range(int(T / dt) + 1):
        t = n * dt
        if n in plot_times:
            u_num_at_t[n] = u.copy()
        
        coeff = fdiv(fmul(mpf(c), mpf(dt)), mpf(dx))
        coeff_squared = fdiv(coeff, coeff)

        for i in range(1, N - 1):
            term2 = fadd(fsub(mpf(u[i + 1]), fmul(mpf(2), mpf(u[i]))), mpf(u[i - 1]))
            update = fmul(coeff_squared, term2)
            u_next[i] = float(fadd(fsub(fmul(mpf(2), mpf(u[i])), mpf(u_prev[i])), update))
        u_next[0] = 0.0
        u_next[-1] = 0.0
        u_prev = u.copy()
        u = u_next.copy()

    # Unified Plot
    fig, axs = plt.subplots(2, 3, figsize=(18, 10), sharey=True)

    # Plot Analytical Solution
    for i, idx in enumerate(plot_times):
        t = idx * dt
        u_analytic = analytical_solution(t)
        axs[0, i].plot(x, u_analytic, 'k-', label=f"Analytical t = {t:.2f}s")
        axs[0, i].set_title(f"Analytical Solution at t = {t:.2f}s")
        axs[0, i].set_xlabel("Position (x) [m]")
        axs[0, i].grid(True)
        axs[0, i].legend()

    # Plot Numerical Solution
    for i, idx in enumerate(plot_times):
        t = idx * dt
        u_num = u_num_at_t[idx]
        axs[1, i].plot(x, u_num, 'r--', label=f"Numerical t = {t:.2f}s")
        axs[1, i].set_title(f"Numerical Solution at t = {t:.2f}s")
        axs[1, i].set_xlabel("Position (x) [m]")
        axs[1, i].grid(True)
        axs[1, i].legend()

    # Set global labels
    fig.supylabel("u(x, t)")
    fig.suptitle(f"Scaled Wave Equation Solutions (Precision: {mp.prec} bits)")
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    # Save the figure
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"wave_solution_bit_depth_{bit_depth}.png")
    plt.savefig(output_file)
    plt.close()

# Loop over bit depths
for bit_depth in range(23, 3, -1):  # From 23 bits to 4 bits
    compute_wave_solution_and_plot(bit_depth)


In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpmath import mp, mpf, fmul, fadd, fsub
import csv

# Constants
L = 100.0                  # Half-length of the domain in meters (standard float)
c = 1.0                    # Wave speed in m/s (standard float)
N = 201                    # Number of spatial points
dx = (2 * L) / (N - 1)     # Spatial step size for [-L, L] domain
dt = dx / c                # Time step size (CFL condition)
T = 1000.0                 # Total simulation time in seconds
x = np.linspace(-L, L, N)  # Spatial grid as a NumPy array
plot_times = [0, int((T / 2) / dt), int(T / dt)]

# Output directories
output_dir = "/Users/danieltompkins/Documents/Devito-UROP-MSci/Scaled-results/"
error_plot_dir = os.path.join(output_dir, "error_plots")
error_data_dir = os.path.join(output_dir, "error_data")
fft_plot_dir = os.path.join(output_dir, "fft_plots")
os.makedirs(error_plot_dir, exist_ok=True)
os.makedirs(error_data_dir, exist_ok=True)
os.makedirs(fft_plot_dir, exist_ok=True)

# Analytical Solution
def analytical_solution(t, x=x, c=c, L=L):
    t_np = float(t)
    return np.cos(2 * np.pi * c * t_np / L) * np.sin(2 * np.pi * x / L)

# Function to compute wave solution and errors at a given precision
def compute_wave_solution_and_errors(bit_depth):
    mp.prec = bit_depth
    u = analytical_solution(0)
    u_prev = u.copy()
    u_next = np.zeros(N, dtype=np.float64)
    u_num_at_t = {}
    errors = []

    for n in range(int(T / dt) + 1):
        t = n * dt
        if n in plot_times:
            u_num_at_t[n] = u.copy()
        
        coeff = fdiv(fmul(mpf(c), mpf(dt)), mpf(dx))
        coeff_squared = fdiv(coeff, coeff)

        for i in range(1, N - 1):
            term2 = fadd(fsub(mpf(u[i + 1]), fmul(mpf(2), mpf(u[i]))), mpf(u[i - 1]))
            update = fmul(coeff_squared, term2)
            u_next[i] = float(fadd(fsub(fmul(mpf(2), mpf(u[i])), mpf(u_prev[i])), update))
        u_next[0] = 0.0
        u_next[-1] = 0.0
        u_prev = u.copy()
        u = u_next.copy()
        u_analytic = analytical_solution(t)
        max_error = max(abs(u[i] - u_analytic[i]) for i in range(N))
        errors.append((t, max_error))

    return u_num_at_t, errors

# Function to generate and save plots and data
def save_results(bit_depth, u_num_at_t, errors):
    # FFT Plot
    u_start = u_num_at_t[plot_times[0]]
    u_final = u_num_at_t[plot_times[2]]
    fft_start = np.fft.fft(u_start)
    fft_final = np.fft.fft(u_final)
    freqs_start = np.fft.fftfreq(len(u_start), d=dt)
    freqs_final = np.fft.fftfreq(len(u_final), d=dt)
    positive_freqs_start = freqs_start[freqs_start >= 0]
    positive_freqs_final = freqs_final[freqs_final >= 0]
    fft_magnitude_start = np.abs(fft_start)[freqs_start >= 0]
    fft_magnitude_final = np.abs(fft_final)[freqs_final >= 0]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.plot(positive_freqs_start, fft_magnitude_start, label="FFT Start")
    ax1.set_title("Scaled Frequency Spectrum at t0")
    ax1.set_xlabel("Frequency (Hz)")
    ax1.set_ylabel("Amplitude")
    ax1.grid(True)
    ax2.plot(positive_freqs_final, fft_magnitude_final, label="FFT End", color="red")
    ax2.set_title("Scaled Frequency Spectrum at tN")
    ax2.set_xlabel("Frequency (Hz)")
    ax2.set_ylabel("Amplitude")
    ax2.grid(True)
    fig.suptitle(f"FFT (Precision {bit_depth} bits)")
    fft_file = os.path.join(fft_plot_dir, f"fft_precision_{bit_depth}.png")
    plt.savefig(fft_file)
    plt.close()

    # Error Plot
    times, max_errors = zip(*errors)
    plt.figure(figsize=(8, 6))
    plt.plot(times, max_errors, label="Max Absolute Error")
    plt.xlabel("Time (s)")
    plt.ylabel("Max Error")
    plt.title(f"Scaled Max Error Over Time (Precision {bit_depth} bits)")
    plt.grid(True)
    error_plot_file = os.path.join(error_plot_dir, f"error_precision_{bit_depth}.png")
    plt.savefig(error_plot_file)
    plt.close()

    # Save error data to CSV
    error_data_file = os.path.join(error_data_dir, f"errors_precision_{bit_depth}.csv")
    with open(error_data_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Time", "Max Error"])
        writer.writerows(errors)

# Main Loop Over Bit Depths
for bit_depth in range(23, 3, -1):  # Loop from 23 bits to 4 bits
    u_num_at_t, errors = compute_wave_solution_and_errors(bit_depth)
    save_results(bit_depth, u_num_at_t, errors)

print(f"Results saved in '{output_dir}' directory.")


Results saved in '/Users/danieltompkins/Documents/Devito-UROP-MSci/Scaled-results/' directory.


In [ ]:
# pull out max error at each bit depth, plot this on the y axis with bit depth on the x